In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import string
import math
import random

In [2]:
def make_dict(data_ru, num=1):
    cv = CountVectorizer(data_ru, ngram_range=(num, num), analyzer='char')
    count_vector = cv.fit_transform(data_ru)

    #dict CountVectorizer
    df = pd.DataFrame(cv.vocabulary_.items())
    df.columns = ['char_val', 'ind']
    cols = df.sort_values(by='ind').char_val.tolist()

    #data
    df_cv = pd.DataFrame(count_vector.todense())
    df_cv.columns = cols
    
    dict_cnt = dict()
    for col in df_cv.columns:
        dict_cnt[col] = df_cv[col].sum()

    df_cnt = pd.DataFrame(dict_cnt.items())
    df_cnt.columns = ['char_val', 'sum']
    df_cnt = df_cnt.sort_values(by='sum', ascending=False)
    return df_cnt

In [3]:
with open('corpora/WarAndPeace.txt', 'r', encoding="utf8") as file:
    data_ru = file.readlines()
    file.close()

In [4]:
list('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')[0:5]

['а', 'б', 'в', 'г', 'д']

Код основан на статье 
via http://probability.ca/jeff/ftpdir/decipherart.pdf

In [28]:
def make_cipher_dict(data_ru):
    dict_cipher = {}
    alphabet = list('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
    #print(type(data_ru))
    for i in range(len(data_ru)):
        dict_cipher[alphabet[i]] = data_ru[i]
    return dict_cipher

def apply_cipher_on_text(text, coding):
    cipher_dict = make_cipher_dict(coding) 
    text = list(text)
    coding_text = ''
    for elem in text:
        if elem.lower() in cipher_dict:
            coding_text += cipher_dict[elem.lower()]
        else:
            coding_text += ' '
    return coding_text

In [37]:
# def make_cnt_dict(path):
#     dict_cnt = {}
#     alphabet_list = list('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
#     with open(path, 'r', encoding="utf8") as file:
#         for line in file:
#             data = list(line.strip())
#             for i in range(len(data)-1):
                
#                 bigram1 = data[i].lower()
#                 bigram2 = data[i+1].lower()
                
#                 if bigram1 not in alphabet_list:
#                     bigram1 = " "
                
#                 if bigram2 not in alphabet_list:
#                     bigram2 = " "
                
#                 bigram = bigram1 + bigram2
                
#                 if bigram in dict_cnt:
#                     dict_cnt[bigram] += 1
#                 else:
#                     dict_cnt[bigram] = 1
#     return dict_cnt


def make_cnt_dict(text):
    dict_cnt = {}
    alphabet = list('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
    data = list(text.strip())
    for i in range(len(data)-1):
        bigram1 = data[i].lower()
        bigram2 = data[i+1].lower()
        if bigram1 not in alphabet:
            bigram1 = " "
        if bigram2 not in alphabet:
            bigram2 = " "
        bigram = bigram1 + bigram2
        if bigram in dict_cnt:
            dict_cnt[bigram] += 1
        else:
            dict_cnt[bigram] = 1
    return dict_cnt

def get_score(text,cipher,scoring_params):
    cipher_dict = make_cipher_dict(cipher)
    decrypted_text = apply_cipher_on_text(text,cipher)
    scored = make_cnt_dict(decrypted_text)
    cipher_score = 0
    for k,v in scored.items():
        if k in scoring_params:
            cipher_score += v * math.log(scoring_params[k])
    return cipher_score

In [38]:
def generate_cipher(cipher):
    pos1 = random.randint(0, len(list(cipher))-1)
    pos2 = random.randint(0, len(list(cipher))-1)
    
    if pos1 == pos2:
        return generate_cipher(cipher)
    else:
        #меняем местами символы
        cipher = list(cipher)
        char1 = cipher[pos1]
        char2 = cipher[pos2]
        
        cipher[pos1] = char2
        cipher[pos2] = char1
        return "".join(cipher)

In [39]:
def MCMC_decrypt(num_iter, cipher_text, params):
    curr_decode = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
    states_set = set()
    best_state = ''
    score = 0
    for i in range(num_iter):
        states_set.add(curr_decode)
        
        new_decode = generate_cipher(curr_decode)
#         print(i, new_decode)
        
        curr_score = get_score(cipher_text, curr_decode, params)
        
        new_score = get_score(cipher_text, new_decode, params)
        
        prob = min(1, math.exp(new_score-curr_score) )
        
        if curr_score > score:
            best_state = curr_decode

        if prob >= random.uniform(0,1):
            curr_decode = new_decode
        
        if i % 500 == 0:
            print("iter",i,":",apply_cipher_on_text(cipher_text,curr_decode)[0:99])
            
    return states_set, best_state

In [11]:
with open('corpora/WarAndPeace.txt', 'r', encoding="utf8") as file:
    data_ru = file.readlines()
    file.close()

In [12]:
# scoring_params = make_cnt_dict('corpora/WarAndPeace.txt')

In [13]:
scoring_params = make_dict(data_ru).set_index('char_val')['sum'].to_dict()

In [13]:
with open('corpora/AnnaKarenina.txt', 'r', encoding="utf8") as file:
    ex = file.readlines()
    file.close()
    
test_ru = ex[30].lower()

In [14]:
plain_text = test_ru

In [15]:
out = ''
for el in np.random.permutation(list('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')):
    out += el
print(out)

чъкязыдцэотсаьмфигщжпюйблврхнёшеу


In [29]:
encryption_key = "бкюэщшпьнуеёйрвлзхдсгымцтжъофчяиа"
cipher_text = apply_cipher_on_text(plain_text, encryption_key)

In [30]:
plain_text

'«анна каренина» поразила современников «вседневностью содержания». необычайная свобода, раскованность повествования удивительно сочетались в этом романе с цельностью художественного взгляда автора на жизнь. он выступал здесь как художник и мыслитель и назначение искусства видел «не в том, чтобы неоспоримо разрешить вопрос, а в том, чтобы заставить любить жизнь в бесчисленных, никогда не истощимых всех ее проявлениях» (61, 100).[1]\n'

In [44]:
out = ''
for el in np.random.permutation(list('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')):
    out += el
print(out)

ньщучеъёжврксшхадиэзбпяйюлцымгтоф


In [156]:
decryption_key = "кдслывшхубфжъйнзёэемюотаищчяьгцрп"

print("Text To Decode:", cipher_text)
print("\n")
states,best_state = MCMC_decrypt(10000,cipher_text,scoring_params)
print("\n")
print("Decoded Text:", apply_cipher_on_text(cipher_text,best_state))
print("\n")
print("MCMC KEY FOUND:", best_state)
print("ACTUAL DECRYPTION KEY:", decryption_key)

Text To Decode:  бввб ёбхшвувб  злхбнуйб длюхшршввуёлю  юдшщвшювлдсчи длщшхьбвуа   вшлкфтбевба дюлклщб  хбдёлюбввлдсч злюшдсюлюбвуа гщуюусшйчвл длтшсбйудч ю яслр хлрбвш д цшйчвлдсчи мгщльшдсюшввлэл юнэйащб бюслхб вб ьунвч  лв юфдсгзбй нщшдч ёбё мгщльвуё у рфдйусшйч у вбнвбтшвуш удёгддсюб юущшй  вш ю слр  тслкф вшлдзлхурл хбнхшжусч юлзхлд  б ю слр  тслкф нбдсбюусч йикусч ьунвч ю кшдтудйшввфм  вуёлэщб вш удслъурфм юдшм шш зхлаюйшвуам                


iter 0 :  бввб ёбхшвувб  злхбнугб длюхшршввуёлю  юдшщвшювлдсчи длщшхьбвуа   вшлкфтбевба дюлклщб  хбдёлюбввлд
iter 500 :  овво тогуваво  пегозако лебгудувватеб  блурвубвелись леругчоваю   вуемнжойвою лбемеро  голтебоввел
iter 1000 :  уссу муносасу  пенузаду левнотоссамев  влорсовселкиь лерончусая   соебыгуйсуя лвеберу  нулмевуссел
iter 1500 :  анна маденуна  чодазула совдереннумов  всетневноский сотедгануя   необыпаьная свобота  дасмованнос
iter 2000 :  анна паренуна  чоразула совреденнупов  всетневноскию сотержануя   необымайная свобота  ра

In [157]:
test_ru

'«анна каренина» поразила современников «вседневностью содержания». необычайная свобода, раскованность повествования удивительно сочетались в этом романе с цельностью художественного взгляда автора на жизнь. он выступал здесь как художник и мыслитель и назначение искусства видел «не в том, чтобы неоспоримо разрешить вопрос, а в том, чтобы заставить любить жизнь в бесчисленных, никогда не истощимых всех ее проявлениях» (61, 100).[1]\n'

In [159]:
test_ru_decoding = apply_cipher_on_text(cipher_text,best_state)
print(len(test_ru_decoding) == len(test_ru))

count = 0
for i in range(len(test_ru)):
    if test_ru[i] == test_ru_decoding[i]:
        count += 1

print('accuracy: ', count/len(test_ru))

True
accuracy:  0.9356321839080459


### Task 4

In [95]:
cipher_text2 = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'

In [96]:
len(cipher_text2)

230

In [98]:
cipher_text2

'←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'

In [102]:
cipher_text2 = cipher_text2.replace('↹', ' ')

In [104]:
orig = make_dict(list(cipher_text2))

In [113]:
orig['enc'] = pd.Series(list(' абвгдежзиклмнопрстуфхцчшщюя'))

In [116]:
coding_dict = orig.set_index('char_val')['enc'].to_dict()

In [117]:
test_ru_coding = ''.join([coding_dict[cipher_text2[i]] for i in range(len(cipher_text2))])
test_ru_coding

'ацте лп лежеза гфчкщтогпм ете юфхзе гфчкщтогпм зарцз у язфвф цффинагед рфзфчпм таврф ючфхезщзо црфчаа лцавф лп лца цжатщте ючщлетогф е юфтухеза кщрцекщтогпм ищтт шщ юфцтажгаа хазлачзфа шщжщгеа ручцщ бфзд рфгахгф д гехавф га фианщс'

In [129]:
states, best_state = MCMC_decrypt(10000,test_ru_coding,scoring_params)

iter 0 : атце лп лежеза гфчкщцогпм еце юфхзе гфчкщцогпм зартз у язфвф тффинагед рфзфчпм цаврф ючфхезщзо трфч
iter 500 : нтал чя члулен иовьсашиях лал зодел иовьсашиях енкте р цеого тообынилю коеовях ангко зводлесеш тков
iter 1000 : ерал ды длулне товьсаятых лал зогнл товьсаятых некрн и жномо рообчетлю коновых аемко звоглнсня рков
iter 1500 : ерла ды даиане товьслятый ала зочна товьслятый непрн у жномо рообгетаю поновый лемпо звочансня рпов
iter 2000 : ерла ды даюате новсульный ала гочта новсульный тепрт и этомо роожценая потовый лемпо гвочатуть рпов
iter 2500 : ерла вы важате нодсильный ала гочта нодсильный тепрт у этомо рообшеная потодый лемпо гдочатить рпод
iter 3000 : ерла вы вазате нодмильный ала гочта нодмильный тепрт у этосо рообщеная потодый леспо гдочатить рпод
iter 3500 : ерли вы вимите нодкальный или гочти нодкальный тепрт у этосо рообщения потодый леспо гдочитать рпод
iter 4000 : ерли вы вимите нодкальный или гочти нодкальный тепрт у этосо рообцения потодый леспо гдочитать р

In [140]:
apply_cipher_on_text(test_ru_coding, 'ехгняиъдтбюмвйщьыкюлуочсрзажфёцпэ')

'если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю'